# Coursera Applied Data Science Capstone Project

Welcome to my coursera Data Science Capstone Project. I am going to create the entire project in this one notebook, so please read all the sections before you grade my notebook for the assignment. This capstone project will be exploring cities and neighborhoods using data to support a decision we will have to make.

## Introduction/Business Problem

A great way to encapsulate all the topics we have learned throughout the IBM Data Science Professional Certificate course is to combine our geographical visualization skills with the ability to request location and venue data from the Foursquare API. In this project, we will be opening a hypothetical restaurant and want to find the best location for this restaurant in some California cities that we have chosen. The restaurant business is very intense in California because it is an area where you can get the best ingredients in the world and there are a lot of customers willing to spend their money on quality food.

The two cities we will be comparing are San Diego and San Francisco. San Diego is home to a burgeoning food scene due to the influx of craft breweries and more sophisticated dining offered throughout the city. San Francisco is a very well established city with many great restaurants that work hard to keep its appeal, but if successful will become powerhouses in the culinary scene. It is our job to determine if opening a small boutique restaurant that serves differing styles of charcuterie plates and complementary beverages will have a great chance of being successful in either the fastly expanding city of San Diego or the established culinary scene of San Francisco.

The stakeholders in this process is a group of investors who will fund the opening of the restaurant and the future owner of the restaurant itself. The stakeholders have much of their investment on the line because the price of real estate between the two cities is very drastic, they will want to make sure that the restaurant they invest in will prove fruitful and give them a return on the investment. The other important stakeholder would be the owner and manager of the restaurant. The owner would want to know if the money that they get from the investors is being utilized efficiently and properly while building and creating the restaurant of their dreams. There is an important decision on the line of where this business will thrive and if it will be profitable.

## Data

There is an enormous amount of data on the internet that we can utilize to help us make the decisions outlined in the Introduction. One major outlet for this data will be Foursquare, which provides numerous amounts of data of venues or restaurants in a specific location that you request. For this particular problem, we will focus on the location data of restaurants, the type of food that is offered, and we will want to look into the density and uniqueness of restaurants within a specific radius.

The restaurant we are making this decision for has a few requirements they want so that there is optimal foot-traffic and availability for the customer. There cannot be other tapas/small snack bars within 100 meters to avoid competition. There must be a heavy flow of pedestrian traffic for more chances of customers. Parking spaces are preferred, however we understand that San Francisco and San Diego can become congested in the more popular parts of the city, so parking is not mandatory. There should be other restaurants and bars within the area because it is most likely that customers will move from one restaurant to the next as they progress through their night. This particular restaurant will want to cater to a snacking and drinking crowd, so there should be sufficient larger and filling food options nearby so that customers are more likely to walk from one business to the next.

We have access to the Foursquare API, giving us large amounts of data on the types of restaurants, what kind of food they offer, and if they are popular with the locals through tips. We will be looking at the density of restaurants within the cities to find a place where there are plenty of restaurants but each with their own style as to not saturate the region with the same types of food. This can be done by clustering neighborhoods of restaurants together and mapping the different types of food options available in the neighborhood. This will be calculated using a diversity index showing the many different types of restaurants within that neighborhood. 

Another dataset we would find useful is the free dataset provided by Yelp that characterizes many metropolitan areas. Our cities that we are interested in are included in this dataset and it provides a lot of information such as cuisine, review ratings, recommendations, and much more.